In [7]:
import math
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import graphviz


In [8]:
x1 = torch.Tensor([2.0]).double();                  x1.requires_grad = True
x2 = torch.Tensor([0.0]).double();                  x2.requires_grad = True
w1 = torch.Tensor([-3.0]).double();                 w1.requires_grad = True
w2 = torch.Tensor([1.0]).double();                  w2.requires_grad = True
b = torch.Tensor([6.8813735870195432]).double();    b.requires_grad = True
n = x1*w1 + x2*w2 + b
o = torch.tanh(n)

print(o.data.item())
o.backward()

print('---')
print('x2', x2.grad.item())
print('w2', w2.grad.item())
print('x1', x1.grad.item())
print('w1', w1.grad.item())

0.7071066904050358
---
x2 0.5000001283844369
w2 0.0
x1 -1.5000003851533106
w1 1.0000002567688737


Lets Build a NN! (Multi Layer Perceptron) 
The immediate class below is a simplified Value class that modifies operators for forward and backprop, which is a spelled out version of the pyTorch code above

In [9]:
class Value:
  
  def __init__(self, data, _children=(), _op='', label=''):
    self.data = data
    self.grad = 0.0
    self._backward = lambda: None
    self._prev = set(_children)
    self._op = _op
    self.label = label

  def __repr__(self):
    return f"Value(data={self.data})"
  
  def __add__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data + other.data, (self, other), '+')
    
    def _backward():
      self.grad += 1.0 * out.grad
      other.grad += 1.0 * out.grad
    out._backward = _backward
    
    return out

  def __mul__(self, other):
    other = other if isinstance(other, Value) else Value(other)
    out = Value(self.data * other.data, (self, other), '*')
    
    def _backward():
      self.grad += other.data * out.grad
      other.grad += self.data * out.grad
    out._backward = _backward
      
    return out
  
  def __pow__(self, other):
    assert isinstance(other, (int, float)), "only supporting int/float powers for now"
    out = Value(self.data**other, (self,), f'**{other}')

    def _backward():
        self.grad += other * (self.data ** (other - 1)) * out.grad
    out._backward = _backward

    return out
  
  def __rmul__(self, other): # other * self
    return self * other

  def __truediv__(self, other): # self / other
    return self * other**-1

  def __neg__(self): # -self
    return self * -1

  def __sub__(self, other): # self - other
    return self + (-other)

  def __radd__(self, other): # other + self
    return self + other

  def tanh(self):
    x = self.data
    t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
    out = Value(t, (self, ), 'tanh')
    
    def _backward():
      self.grad += (1 - t**2) * out.grad
    out._backward = _backward
    
    return out
  
  def exp(self):
    x = self.data
    out = Value(math.exp(x), (self, ), 'exp')
    
    def _backward():
      self.grad += out.data * out.grad # NOTE: in the video I incorrectly used = instead of +=. Fixed here.
    out._backward = _backward
    
    return out
  
  
  def backward(self):
    
    topo = []
    visited = set()
    def build_topo(v):
      if v not in visited:
        visited.add(v)
        for child in v._prev:
          build_topo(child)
        topo.append(v)
    build_topo(self)
    
    self.grad = 1.0
    for node in reversed(topo):
      node._backward()

In [27]:

class Neuron:
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))

    def __call__(self, x):
        #forward pass of neuron 
        act = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out
        
    def parameters(self):
      return self.w + [self.b]


class Layer:
  
  def __init__(self, nin, nout):
    self.neurons = [Neuron(nin) for _ in range(nout)]
  
  def __call__(self, x):
    outs = [n(x) for n in self.neurons]
    return outs[0] if len(outs) == 1 else outs
  
  def parameters(self):
    return [p for neuron in self.neurons for p in neuron.parameters()]    



class MLP:
  
  def __init__(self, nin, nouts):
    sz = [nin] + nouts
    self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
  
  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x
  
  def parameters(self):
    return [p for layer in self.layers for p in layer.parameters()]



Value(data=0.7552200712788215)

In [28]:
n.parameters()

[Value(data=-0.2983336228789246),
 Value(data=0.9901366977650965),
 Value(data=-0.9793253412324325),
 Value(data=0.30091696521902156),
 Value(data=-0.1769218628006468),
 Value(data=0.148314853012961),
 Value(data=-0.0808128705141562),
 Value(data=0.17335903220956506),
 Value(data=0.2970741883972561),
 Value(data=-0.5620666702844466),
 Value(data=0.9329899154979326),
 Value(data=0.11697507337959401),
 Value(data=0.6633182136019113),
 Value(data=0.3081048732362639),
 Value(data=-0.5651022921642548),
 Value(data=-0.902642943441635),
 Value(data=-0.2532780773539596),
 Value(data=-0.31270112479565215),
 Value(data=0.17352752043600583),
 Value(data=0.22637552500404134),
 Value(data=-0.6650260328998521),
 Value(data=-0.20249146528733797),
 Value(data=0.43767085602959455),
 Value(data=-0.7359949808736117),
 Value(data=0.6404022402796419),
 Value(data=-0.09071173613119954),
 Value(data=-0.6493647496225257),
 Value(data=-0.5003478935091681),
 Value(data=0.12501649143449955),
 Value(data=-0.06452

In [71]:
#This code constructs the MLP and does forward prop
x = [2.0, 3.0, -0.1]
n = MLP(3,[4,4,1])
n(x)

Value(data=0.7235951253685742)

In [72]:
#Backprop portion (Training)
xs  = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] #These are the desired targets


In [75]:
for k in range(20):

#forward pass
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout in zip(ys, ypred))

    #backward pass
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()


    #update (the -0.05 here is the learning rate/ step size) also considered stochastic gradient descent
    for p in n.parameters():
        p.data += -0.05 * p.grad 

    print(k, loss.data)


0 0.03596370572304858
1 0.0338194967603944
2 0.031904585032209665
3 0.030184951786939047
4 0.028632885445086194
5 0.027225600998057122
6 0.025944203729463357
7 0.02477290176220452
8 0.023698400876496273
9 0.022709434526769372
10 0.021796395308970654
11 0.02095104337533677
12 0.020166273796905332
13 0.019435929506322572
14 0.01875464979215212
15 0.018117746748743734
16 0.017521103876863356
17 0.016961092361770885
18 0.01643450155424473
19 0.01593848093574162


Now lets do gradient descent manually!

In [76]:
ypred
#WOW! look at those predictions!

[Value(data=0.9645639808810276),
 Value(data=-0.9562605560037697),
 Value(data=-0.9207423142112285),
 Value(data=0.9194528103094189)]

The following is some code to visualize the neural nets we build!

In [11]:
import graphviz
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot